# Dimension Reduction and Clustering Analysis
### By: Swati Kohli, Poonam Patil

### Introduction
**The project is to predict the house sale price. There are two methods explored to perform prediction.**

**Part 1- Dimension reduction:**  
The method used in this part involves using different dimension reduction techniques for numerical and categorical featues and building linear regression model with ridge regularization. The model results are then compared with base model to test the performance.

**Part 2: Clustering Analysis**  
The method in this part clusterings analysis with numerical and categorical data combined. Gower distance metric is  used to measure the distance. The Gower distance is essentially a special distance metric that measures numerical data and categorical data separately, then combine them to form a distance calculation.

Data for the project is taken from below website:

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview

## Part 1 : Dimension reduction

In [1]:
# Import all required libraries
import numpy as np
import pandas as pd
import prince

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.linear_model import LinearRegression
import itertools
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics

### 1. Load the data

In [2]:
# Load data set
data = pd.read_csv('train.csv')
data = data.drop('Id', axis = 1)

# Remove columns that have too many missing values
data = data.drop(data.columns[data.isnull().sum() > 30], axis = 1)

# Remove missing values
data.dropna(inplace = True)

In [3]:
data.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [4]:
X = data.copy()
del X['SalePrice']
y = data['SalePrice']

#### Split data into train_valid and test set

In [5]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y , test_size = 0.3, random_state = 1)

#### Further split train_valid dataset into train and valid set for hyperparameter tuning

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2 , random_state=1)

### 2. Pre-processing the train and valid data

In [7]:
# separate numerical and categorical features into different datasets
X_train_num = X_train.select_dtypes(include=[np.number])
X_train_cat = X_train.select_dtypes(exclude=[np.number])

X_valid_num = X_valid.select_dtypes(include=[np.number])
X_valid_cat = X_valid.select_dtypes(exclude=[np.number])

In [8]:
# to make sure the training feature and testing feature has same number of levels
keep = X_train_cat.nunique() == X_valid_cat.nunique()
X_train_cat = X_train_cat[X_train_cat.columns[keep]]
X_valid_cat =X_valid_cat[X_valid_cat.columns[keep]]

In [9]:
# For categorical features that have same levels, make sure the classes are the same
keep = []
for i in range(X_train_cat.shape[1]):
    keep.append(all(np.sort(X_train_cat.iloc[:,i].unique()) == np.sort(X_valid_cat.iloc[:,i].unique())))
X_train_cat = X_train_cat[X_train_cat.columns[keep]]
X_valid_cat = X_valid_cat[X_valid_cat.columns[keep]]

In [10]:
#create hyperparameter set
alphas = np.logspace(-2,2,5)            # parameter in ridge regression
n_pca_components =[25,30,34]            # for n_components parameter in KernelPCA
kernels= ['rbf', 'sigmoid']             # wrapper for rbf and sigmoid KernelPCA model
gammas = np.linspace(0.03, 0.05, 3)     # parameter in KernelPCA
n_mca_components = [5,7,9,11]           # for n_components parameter in MCA
n_iters = np.logspace(1,2,3, dtype=int) # for n_iter parameter in MCA

hyperparam_set = list(itertools.product(alphas, n_pca_components, kernels, gammas, n_mca_components, n_iters))

In [11]:
# scale numerical data before applying PCA       
scaler= StandardScaler() 
scaler.fit(X_train_num)      # fit scaler on train set
X_train_num = pd.DataFrame(scaler.transform(X_train_num)) # transform train set
X_valid_num = pd.DataFrame(scaler.transform(X_valid_num)) # transform validation set

### 3. Building a wrapper to select the best model and best hyperparameter set

In [12]:
valid_score =[]     # initialize a list to store validation score

for params in hyperparam_set:
    
    # train KernelPCA on numerical data
    pca = KernelPCA(n_components = params[1], kernel=params[2], gamma=params[3], fit_inverse_transform = True) 
    X_train_pca = pd.DataFrame(pca.fit_transform(X_train_num))
    X_valid_pca = pd.DataFrame(pca.transform(X_valid_num))
    
    # train MCA on categorical data
    mca = prince.MCA(n_components=params[4], n_iter=params[5], copy=True, check_input=True, benzecri=False, 
                 random_state= 1, engine='auto')
    mca=mca.fit(X_train_cat)        # fit MCA on X_train_cat data
    
    X_train_mca = mca.transform(X_train_cat)     # tranform X_train_cat data on MCA
    X_valid_mca = mca.transform(X_valid_cat)     # tranform X_valid_cat data on MCA
    
    # StandardScaler reset the row indexes. 
    # Therefore we need to reset the index for MCA components too. As we need to combine these two datasets (PCA+MCA)
    X_train_mca = X_train_mca.reset_index(drop=True)
    X_valid_mca = X_valid_mca.reset_index(drop=True)
    
    # combine PCA and MCA to form complete training dataset
    X_train = pd.concat([X_train_pca, X_train_mca], axis=1, ignore_index=True)
    X_valid = pd.concat([X_valid_pca, X_valid_mca], axis=1, ignore_index=True)
    
    # fit linear regression with Ridge regularization on dimensally reduced components (i.e. PCA+MCA)
    lm= linear_model.Ridge(alpha=params[0])
    lm.fit(X_train, y_train)
    valid_score.append(metrics.mean_squared_error(lm.predict(X_valid), y_valid))

print('best hyperparameter set:', hyperparam_set[np.argmin(valid_score)])
print('MSE on validation set:', min(valid_score))

best hyperparameter set: (0.1, 34, 'sigmoid', 0.03, 11, 10)
MSE on validation set: 985171890.5521743


**Observation: The best model selected is sigmoid kernel basis function for dimension reduction. All PCA and MCA components are selected in parameter tuning. Best gamma is 0.03 and best n_iters =10**

In [13]:
best_hyperparam = hyperparam_set[np.argmin(valid_score)]

In [14]:
# storing best hypeerparameters into variables to be used in training the final model
best_alpha = best_hyperparam[0]
best_n_pca_components = best_hyperparam[1]
best_kernel = best_hyperparam[2]
best_gamma = best_hyperparam[3]
best_n_mca_components = best_hyperparam[4]
best_n_iters = best_hyperparam[5]

### 4. Training the model on train_valid set with best model and hyperparameters selected

#### Preprocessing train_valid dataset

In [15]:
# separate numerical and categorical features into different datasets
X_train_valid_num = X_train_valid.select_dtypes(include=[np.number])
X_train_valid_cat = X_train_valid.select_dtypes(exclude=[np.number])

X_test_num = X_test.select_dtypes(include=[np.number])
X_test_cat = X_test.select_dtypes(exclude=[np.number])

In [16]:
# to make sure the training feature and testing feature has same number of levels
keep = X_train_valid_cat.nunique() == X_test_cat.nunique()
X_train_valid_cat = X_train_valid_cat[X_train_valid_cat.columns[keep]]
X_test_cat =X_test_cat[X_test_cat.columns[keep]]

In [17]:
# For categorical features that have same levels, make sure the classes are the same
keep = []
for i in range(X_train_valid_cat.shape[1]):
    keep.append(all(np.sort(X_train_valid_cat.iloc[:,i].unique()) == np.sort(X_test_cat.iloc[:,i].unique())))
X_train_valid_cat = X_train_valid_cat[X_train_valid_cat.columns[keep]]
X_test_cat = X_test_cat[X_test_cat.columns[keep]]

In [18]:
# scale numerical data before applying PCA        
scaler= StandardScaler() 
scaler.fit(X_train_valid_num)      # fit scaler on train set
X_train_valid_num = pd.DataFrame(scaler.transform(X_train_valid_num)) # transform train set
X_test_num = pd.DataFrame(scaler.transform(X_test_num)) # transform validation set

### 5. Employing KernelPCA and MCA dimention reduction techniques on train_valid dataset

In [19]:
# train PCA on numerical data
pca = KernelPCA(n_components = best_n_pca_components, kernel=best_kernel, gamma=best_gamma, fit_inverse_transform = True) 
X_train_valid_pca = pd.DataFrame(pca.fit_transform(X_train_valid_num))
X_test_pca = pd.DataFrame(pca.transform(X_test_num))

# train MCA on categorical data
mca = prince.MCA(n_components=best_n_mca_components, n_iter= best_n_iters, copy=True, check_input=True, benzecri=False, 
             random_state= 1, engine='auto')
mca=mca.fit(X_train_valid_cat)

X_train_valid_mca = mca.transform(X_train_valid_cat)
X_test_mca  = mca.transform(X_test_cat)

## StandardScaler reset the row indexes for numerical data. Therefore reset the index MCA components too
# reset index of MCA components
X_train_valid_mca = X_train_valid_mca.reset_index(drop=True)
X_test_mca = X_test_mca.reset_index(drop=True)

# combine PCA and MCA to form complete training dataset
X_train_valid = pd.concat([X_train_valid_pca, X_train_valid_mca], axis=1, ignore_index=True)
X_test = pd.concat([X_test_pca, X_test_mca], axis=1, ignore_index=True)

### 6. Fitting linear regression with ridge regularization on train_valid dataset with best hyperparameters selcted.

In [20]:
# fit linear regression with Ridge regularization on dimensally reduced components (i.e. PCA+MCA)
lm= linear_model.Ridge(alpha=best_alpha)
lm.fit(X_train_valid, y_train_valid)
test_score = metrics.mean_squared_error(lm.predict(X_test), y_test)
print('Test MSE:', test_score)

Test MSE: 947741457.1330783


### 7. Testing results with base model

In [21]:
# Creating dummy variables for the categorical variables in train_valid dataset
X_train_valid_dummy = pd.get_dummies(X_train_valid_cat, drop_first=True)
X_train_valid_dummy.reset_index(drop=True, inplace=True)

In [22]:
# Creating dummy variables for the categorical variables in test dataset
X_test_dummy = pd.get_dummies(X_test_cat, drop_first = True)
X_test_dummy.reset_index(drop=True, inplace=True)

In [23]:
# concatenate numerical and dummy features to create a data for modelling
X_train_valid = pd.concat([X_train_valid_num, X_train_valid_dummy], axis=1, ignore_index=True)
X_test = pd.concat([X_test_num, X_test_dummy], axis=1, ignore_index=True)

In [24]:
# Train linear regression model with Ridge regularization
lm_reg= linear_model.Ridge(alpha=best_alpha)
lm_reg.fit(X_train_valid, y_train_valid)   # fit model on train_valid set
test_score = metrics.mean_squared_error(lm_reg.predict(X_test), y_test)  # test model on test set
print('Test MSE with base model:', test_score)   

Test MSE with base model: 1003333118.03285


### 8. Comparison of results

From the results we can see that, in the linear regression with dimension reducted components, best MSE on test set is achieved when the model used all of the PCA and MCA components. However, the results with a linear model using sigmoid kernel basis function for dimension reducition gives best prediction of house sale price than the simple linear regression with ridge regularization.

## Part II : Clustering Analysis

Clustering analysis can be performed with numerical and categorical data combined. Distance metric is used for the same. Further, Gower distance is used which is a special distance metric that measures numerical data and categorical data separately,then combine them to form a distance calculation.

**Approach:**  
Seperate predictor and response variables (X and y) from the dataset.
1. Install and import Gower Library for mixed data types.
2. Compute the Gower distance of the full predictors set
3. Apply K-medoids using the gower distance matrix as input to get clusters and medoids(index).
4. Compare the clustering result with the 'ground truth'.  
    i. Get array of cluster membership of each observation.  
    ii. Ground Truth = Bin the response variable (of the original data set) into the number of categories as per k clusters.  
    iii. For Clustering performance, compute the normalized mutual information (NMI) between clustering results and the binned categories.

### Step 1

In [25]:
# Step 1. 
# pip install gower
import gower

### Step 2

In [26]:
# Step 2.
# Find the Gower distance matrix of the full predictors set
gd_matrix = gower.gower_matrix(X)
gd_matrix

array([[0.        , 0.24024484, 0.03944052, ..., 0.19145243, 0.25093865,
        0.20125039],
       [0.24024484, 0.        , 0.24959742, ..., 0.24499281, 0.19383048,
        0.1789145 ],
       [0.03944052, 0.24959742, 0.        , ..., 0.20383762, 0.2743255 ,
        0.22117272],
       ...,
       [0.19145243, 0.24499281, 0.20383762, ..., 0.        , 0.25380814,
        0.25413764],
       [0.25093865, 0.19383048, 0.2743255 , ..., 0.25380814, 0.        ,
        0.16829133],
       [0.20125039, 0.1789145 , 0.22117272, ..., 0.25413764, 0.16829133,
        0.        ]], dtype=float32)

### Step 3
#### Let's take k = 5 clusters. Hierachical clustering can also be done for deciding k as a starting point.

In [27]:
# Step 3.
# pip install pyclustering
from pyclustering.cluster.kmedoids import kmedoids

# To use the K-medoid function, provide some initial centers. Let's take k =5. 

# Randomly sample 5 observations as centers.
np.random.seed(50)
k = 5
center_index = np.random.randint(0, len(y), k)
print("Initial centers: ",center_index)

# Create K-Medoids algorithm for processing distance matrix instead of points
kmedoids_instance = kmedoids(gd_matrix, center_index, data_type='distance_matrix')

# Run cluster analysis and obtain results
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters() 
medoids = kmedoids_instance.get_medoids() # new medoids
print("New Medoids are:",medoids)

# Cluster output is given as nested list of clusters(k)
print("/nCluster output example",clusters[0])

Initial centers:  [ 109 1313  132   70  229]
New Medoids are: [712, 210, 1242, 793, 729]
/nCluster output example [712, 7, 10, 34, 37, 45, 55, 78, 81, 91, 97, 101, 102, 109, 113, 116, 124, 135, 144, 152, 166, 171, 187, 194, 196, 205, 209, 215, 225, 227, 232, 234, 244, 252, 264, 268, 272, 293, 297, 298, 312, 318, 329, 338, 355, 362, 363, 367, 386, 415, 421, 424, 428, 429, 444, 454, 463, 470, 474, 482, 497, 499, 504, 514, 515, 528, 558, 568, 580, 585, 592, 603, 609, 616, 619, 623, 630, 645, 652, 659, 664, 670, 676, 692, 698, 706, 710, 714, 725, 730, 738, 740, 754, 763, 767, 772, 774, 777, 782, 788, 792, 807, 815, 822, 827, 834, 839, 842, 850, 854, 855, 856, 859, 881, 886, 888, 889, 892, 939, 940, 941, 945, 950, 957, 965, 986, 988, 991, 997, 999, 1011, 1018, 1035, 1038, 1039, 1042, 1047, 1061, 1067, 1077, 1093, 1098, 1100, 1105, 1153, 1155, 1162, 1180, 1187, 1199, 1215, 1217, 1223, 1236, 1240, 1243, 1264, 1265, 1269, 1270, 1273, 1274, 1278, 1282, 1285, 1287, 1301, 1302, 1307, 1328, 1338, 

**Observation: Labels are not produced. Instead, clusters are made as a list which has index of observations that belong to the respective cluster.**

### Step 4

In [28]:
# Step 4.

# Step 4.i.
# Assign labels to Clusters
cluster_labels = np.zeros([X.shape[0]], dtype=int)
for i in range(1,k):
    cluster_labels[clusters[i]] = i

cluster_labels #array that records the cluster membership of each observation.

array([4, 2, 4, ..., 4, 2, 2])

In [29]:
# Step 4.ii.
# Bin response variable so each group has roughly the same number of observations.
response_binned = pd.qcut(y, q = k, labels = range(k))
response_binned[:10]

0    3
1    3
2    3
3    1
4    4
5    1
6    4
7    3
8    1
9    0
Name: SalePrice, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

Bins are made on the basis of increasing selling price range
(34899.999, 124000.0] < (124000.0, 147000.0] < (147000.0, 179000.0] < (179000.0, 230000.0] < (230000.0, 755000.0]

### Check performance with NMI performance metrics.

In [30]:
# Step4.iii.
# NMI between clustering results and the binned categories.
from sklearn.metrics.cluster import normalized_mutual_info_score

NMI_clusters5 = normalized_mutual_info_score(response_binned, cluster_labels)
print("NMI score with clusters/K = 5:", NMI_clusters5)

NMI score with clusters/K = 5: 0.24936514664331721


**Observation:**  
Given the 'ground truth' class assignments of the samples, NMI is used to evaluate the performance of the clustering technique by measuring the similarity between the results of clustering and 'ground truth'. In this case, ground truth is taken as binned response variable(using pandas.qcut function so each group has roughly the same number of observations).  

Low NMI indicates classes members are predominantly split across different clusters.

####  Since, NMI is normalized, we can use it to compare clusterings with different numbers of clusters. 
### Let's compare different K values(clusters) to see the change in results.
#### K values used for the analysis are 4,3 and 2.

### Clusters 4

In [31]:
# Step 3.

# Randomly sample 4 observations as initial centers.
np.random.seed(50)
k = 4
center_index = np.random.randint(0, len(y), k)
print("Initial centers: ",center_index)

# create K-Medoids algorithm for processing distance matrix instead of points
kmedoids_instance = kmedoids(gd_matrix, center_index, data_type='distance_matrix')

# run cluster analysis and obtain results
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters() 
medoids = kmedoids_instance.get_medoids() # new medoids
print("New Medoids are:",medoids)


# Step 4.

# Step 4.i.
# cluster labels
cluster_labels = np.zeros([X.shape[0]], dtype=int)
for i in range(1,k):
    cluster_labels[clusters[i]] = i

# Step 4.ii.
# Bin response variable to roughly so each group has roughly the same number of observations.
response_binned = pd.qcut(y, q = k, labels = range(k))

# Step4.iii.
# NMI between clustering results and the binned categories.
NMI_clusters4 = normalized_mutual_info_score(response_binned, cluster_labels)
print("NMI score with clusters/K = 4:", NMI_clusters4)

Initial centers:  [ 109 1313  132   70]
New Medoids are: [216, 1242, 1073, 168]
NMI score with clusters/K = 4: 0.2571355084670131


### Clusters 3

In [32]:
# Step 3.

# Randomly sample 4 observations as initial centers.
np.random.seed(50)
k = 3
center_index = np.random.randint(0, len(y), k)
print("Initial centers: ",center_index)

# create K-Medoids algorithm for processing distance matrix instead of points
kmedoids_instance = kmedoids(gd_matrix, center_index, data_type='distance_matrix')

# run cluster analysis and obtain results
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters() 
medoids = kmedoids_instance.get_medoids() # new medoids
print("New Medoids are:",medoids)


# Step 4.

# Step 4.i.
# cluster labels
cluster_labels = np.zeros([X.shape[0]], dtype=int)
for i in range(1,k):
    cluster_labels[clusters[i]] = i

# Step 4.ii.
# Bin response variable to roughly so each group has roughly the same number of observations.
response_binned = pd.qcut(y, q = k, labels = range(k))

# Step4.iii.
# NMI between clustering results and the binned categories.
NMI_clusters3 = normalized_mutual_info_score(response_binned, cluster_labels)
print("NMI score with clusters/K = 3:", NMI_clusters3)

Initial centers:  [ 109 1313  132]
New Medoids are: [602, 1242, 1446]
NMI score with clusters/K = 3: 0.27257573457056755


### Clusters 2

In [33]:
# Step 3.

# Randomly sample 4 observations as initial centers.
np.random.seed(50)
k = 2
center_index = np.random.randint(0, len(y),k)
print("Initial centers: ",center_index)

# create K-Medoids algorithm for processing distance matrix instead of points
kmedoids_instance = kmedoids(gd_matrix, center_index, data_type='distance_matrix')

# run cluster analysis and obtain results
kmedoids_instance.process()

clusters = kmedoids_instance.get_clusters() 
medoids = kmedoids_instance.get_medoids() # new medoids
print("New Medoids are:",medoids)


# Step 4.

# Step 4.i.
# cluster labels
cluster_labels = np.zeros([X.shape[0]], dtype=int)
for i in range(1,k):
    cluster_labels[clusters[i]] = i

# Step 4.ii.
# Bin response variable to roughly so each group has roughly the same number of observations.
response_binned = pd.qcut(y, q = k, labels = range(k))

# Step4.iii.
# NMI between clustering results and the binned categories.
NMI_clusters2 = normalized_mutual_info_score(response_binned, cluster_labels)
print("NMI score with clusters/K = 2:", NMI_clusters2)

Initial centers:  [ 109 1313]
New Medoids are: [729, 1242]
NMI score with clusters/K = 2: 0.39421252800175177


**Observation:**  
K = 2 clusters has highest NMI of 0.394 out of all and k = 5 has the lowest NMI of 0.249.
As the number of K reduces, the NMI increases.

NMI is a good measure for determining the quality of clustering.  
It is an external measure because we need the class labels of the instances to determine the NMI.  
Additionally, if the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient, Calinski-Harabasz index are examples of such an evaluation, where a higher score relates to a model with better defined clusters. 

References:  
    https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html  
    https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-clustering-1.html  
    https://course.ccs.neu.edu/cs6140sp15/7_locality_cluster/Assignment-6/NMI.pdf  
    https://scikit-learn.org/stable/modules/clustering.html  